In [ ]:
import os
import arcpy
import pandas as pd

arcpy.env.overwriteOutput = True
sistema_ctm12 = r"C:\PUBLIC\SERESCO\Resultados\_8_Py\ctm12.prj"

ruta_bd_modelo_fisico = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_7_GDBs\BD_Modelo_Fisico_V0.gdb"
nombre_dataset_insumos_diagnostico = 'Insumos_CAR'

lista_bd_fuente_insumo = [ # ** BD Originales POMCAS
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Alto_Suarez.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Bogota.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Carare_Minero.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Garagoa.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Guavio.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Guayuriba.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Medio_Bajo_Suarez.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Negro.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Seco.gdb',
                    r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Rio_Sumapaz.gdb',
                     # ** BD POMCA Unificado
                    # r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\gdb_pomcas\POMCA_Unificada.gdb',
                     # ** Inventario Fuentes Secundarias
                     # r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\BackUP_PGAR_Inventario\PGAR_Inventario.gdb',
                     # ** Inventario Shape Originales
                     r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_7_GDBs\capas_shp_car.gdb',
                     # ** Inventario Shape Modelo Lógico
                     # r'D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_7_GDBs\capas_pomca_analisis.gdb',
                     # 15:r'C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\[sa] - BD_Indicadores.sde'
                    ]

In [ ]:
ruta_xlsx_diccionario_capas_insumo = r"C:\PUBLIC\PGAR\Resultados\2.Py\PGAR_Calidad\diccionario_insumos_car.xlsx"
diccionario_capas_insumo = pd.read_excel(ruta_xlsx_diccionario_capas_insumo,sheet_name='Hoja 2')

diccionario =  dict([(i,[a,b,c]) for i,a,b,c in zip(diccionario_capas_insumo['llave'], diccionario_capas_insumo['nombre_base'], diccionario_capas_insumo['nombre_dataset'], diccionario_capas_insumo['nombre_capa'])])

In [ ]:
def copia_capa_diccionario_insumo(capa_a_copiar):
    # ? Listado de BD de Análisis Insumos
    for bd in lista_bd_fuente_insumo:
        # ? Recorrido por diccionario con capas
        for llave, valor in diccionario.items():
            nombre_base_datos = os.path.basename(bd)
            base_datos_sin_extension = nombre_base_datos.split('.')[0]
            if valor[0] == nombre_base_datos and valor[1] != 'Sin DataSet' and valor[2] == capa_a_copiar:
                ruta_capas_copia_origen = os.path.join(bd,valor[1],valor[2])
                ruta_capa_copia_salida = os.path.join(ruta_bd_modelo_fisico ,base_datos_sin_extension+'_'+valor[2]+'__SP')
                arcpy.management.Copy(ruta_capas_copia_origen, ruta_capa_copia_salida)
                print(f"Se copia capa {valor[2]}")
            if valor[0] == nombre_base_datos and valor[1] == 'Sin DataSet' and valor[2] == capa_a_copiar:
                ruta_capas_copia_origen = os.path.join(bd,valor[2])
                ruta_capa_copia_salida = os.path.join(ruta_bd_modelo_fisico, base_datos_sin_extension+'_'+valor[2]+'__SP')
                arcpy.management.Copy(ruta_capas_copia_origen, ruta_capa_copia_salida)
                print(f"Se copia capa {valor[2]}")
            else:
                pass
                
    arcpy.env.workspace = ruta_bd_modelo_fisico
    for capa in arcpy.ListFeatureClasses():
        nombre_capa_proyeccion = os.path.basename(capa)
        nombre_capa_proyeccion_estandarizada = nombre_capa_proyeccion.split("__")[0]
        nombre_ruta_salida = os.path.join(ruta_bd_modelo_fisico,nombre_dataset_insumos_diagnostico,nombre_capa_proyeccion_estandarizada)
        arcpy.management.Project(capa, nombre_ruta_salida, sistema_ctm12)
        print(f"Se proyecta y migra capa {capa}")

    for capa in arcpy.ListFeatureClasses():
        arcpy.management.Delete(capa)
        print(f"Se borra capa {capa}")

In [ ]:
feature_diccionario = []
nombre_original_diccionario = []
ruta_comparacion = os.path.join(ruta_bd_modelo_fisico, nombre_dataset_insumos_diagnostico)    
for llave, valor in diccionario.items():
    feature_diccionario_name = valor[0].split('.')[0] + '_' + str(valor[2])
    feature_diccionario_ruta = os.path.join(ruta_comparacion, feature_diccionario_name)
    feature_original = str(valor[2])
    nombre_original_diccionario.append(feature_original)
    feature_diccionario.append(feature_diccionario_ruta)
    
lista_unificada = list(zip(feature_diccionario, nombre_original_diccionario))

for i in lista_unificada:
    if arcpy.Exists(i[0]):
        print(f"La capa {os.path.basename(i[0])} existe")
    else:
        copia_capa_diccionario_insumo(i[1])

In [ ]:
print("FINALIZA PROCESO")